<h1>Data Retrieval</h1>
<p>
the dataset are manually encoded questionnaire from various math resources such as DepEd produced modules posted online, Grade 10 Math Books, mathematics dataset
</p>
<h5>Resources:</h5>
<ul>
    <li>
        DepEd Modules:
        <ul>
            <li></li>
            <li></li>
            <li></li>
        </ul>
    </li>
    <li>Spiral Math 10</li>
    <li></li>
</ul>

In [1]:
import pandas as pd

# df1 = dataframe of manually encoded datasets.

df1 = pd.read_csv('dataset-december.csv', encoding='utf-8')
df1 = df1.drop(columns = ['id'])
df1

,question,label
0,It is the sum of the terms of a sequence.,series_and_sequence
1,Find the sum of the first ten terms of the ari...,series_and_sequence
2,Find the sum of the first 25 terms of the arit...,series_and_sequence
3,The sum of the first 10 terms of an arithmetic...,series_and_sequence
4,The third term of an arithmetic sequence is −1...,series_and_sequence
...,...,...
840,Determine the zeros of the polynomial function...,polynomial_equation
841,Determine the zeros of the polynomial function...,polynomial_equation
842,Determine the zeros of the polynomial function...,polynomial_equation
843,Determine the zeros of the polynomial function...,polynomial_equation


In [2]:
# display instances
df1.label.value_counts()

series_and_sequence    318
polynomials            318
polynomial_equation    209
Name: label, dtype: int64

Adding polynomial_equation data to supply 109 instances of question from mathematics dataset.  

In [3]:
#df2 additional dataset for polynomial equations

df2 = pd.DataFrame(columns=['question', 'label'])
with open("algebra__polynomial_roots.txt", "r") as data:
    #First 109 instance of Dataset      
    list_item = data.readlines()[:217]
    for item in range(0,len(list_item),2):
        df2 = df2.append({"question":list_item[item], "label":"polynomial_equation"}, ignore_index=True)
    data.close()

initial_dataset = df1.append(df2, ignore_index=True)
print('DATASET PREVIEW')
print(initial_dataset)
initial_dataset.to_csv("initial_dataset.csv", encoding ='utf-8') 

DATASET PREVIEW
                                              question                label
0            It is the sum of the terms of a sequence.  series_and_sequence
1    Find the sum of the first ten terms of the ari...  series_and_sequence
2    Find the sum of the first 25 terms of the arit...  series_and_sequence
3    The sum of the first 10 terms of an arithmetic...  series_and_sequence
4    The third term of an arithmetic sequence is −1...  series_and_sequence
..                                                 ...                  ...
949                Factor -35*k**2 + 27415*k - 7830.\n  polynomial_equation
950       Factor -6*n**4/5 - 116*n**3/5 - 38*n**2/5.\n  polynomial_equation
951  What is w in 3*w**3 - 3561*w**2 + 1058505*w - ...  polynomial_equation
952    Find v, given that 12*v**2 - 585*v - 147 = 0.\n  polynomial_equation
953  Determine z so that 3*z**3 + 234*z**2/5 + 864*...  polynomial_equation

[954 rows x 2 columns]


re-sampled the data using mathematics dataset 

In [4]:
print('DATASET INFORMATION')
print(initial_dataset.label.value_counts())

DATASET INFORMATION
series_and_sequence    318
polynomials            318
polynomial_equation    318
Name: label, dtype: int64


<h1>Data Cleaning</h1>

To generalize a formula, and expressions, equation, turn them into one single format
example: 

Math Expression:

$$a_n+\frac{\left(-1\right)^n}{n^2}$$ 


In LaTex (How it is encoded on the Dataset)
<br>
<br>
<br>
<center>\$a_n+\frac{\left(-1\right)^n}{n^2}\$</center>
<br>
<br>
<br>
In Freeform formula (Converted via library Sympy): (It rearrange the solution into PEMDAS format)
<br>
<br>
<br>
<center>(-1)**n/(n**2) + a_n</center>

Sympy Library converts freeform into expression tree which arranges the expression on how it will be processed if solved. so similarly built expressions can be counted as one token.

In [5]:
import re

def change_to_dollar(exp):
    exp = re.sub("\.\.\.|\\dots|…","x",exp)
    exp = re.sub("=","$ $",exp)
    return exp

for i in range(len(initial_dataset)):
    qs = initial_dataset.loc[i, 'question']
    fx = re.search('[$].+?[=].+?[$]',qs)
    len_check = len(re.findall('[$].+?[=].+?[$]', qs))
    iplus = 0
    while iplus<len_check:
        iplus += 1
        qs = qs[:fx.span()[0]] + " " + change_to_dollar(fx.group()) + " " + qs[fx.span()[1]:]
        #check for the next LaTex Enclosure if any         
        fx = re.search('[$].+?[=].+?[$]', qs)
    #the new question is added to index i       
    initial_dataset.loc[i,'question'] = qs
    

# Since Sympy cannot read sqrt latex we'll convert it into exponent format using regex
pattern = r"\\sqrt\{(.+?)\}"
replacement = r"(\1)^{\\frac{1}{2}}"

dataset_sympytolatex = initial_dataset.replace(pattern, replacement, regex=True)

print("dataset preview ready for converting LaTeX expression enclosed in $$ into freeform expression")
dataset_sympytolatex.head()

dataset preview ready for converting LaTeX expression enclosed in $$ into freeform expression


,question,label
0,It is the sum of the terms of a sequence.,series_and_sequence
1,Find the sum of the first ten terms of the ari...,series_and_sequence
2,Find the sum of the first 25 terms of the arit...,series_and_sequence
3,The sum of the first 10 terms of an arithmetic...,series_and_sequence
4,The third term of an arithmetic sequence is −1...,series_and_sequence


In [6]:
from latex2sympy2 import latex2sympy
import re

# copy pasted from previous code so I just reassigned the old used df so I have minimal changes to the dataset
df = dataset_sympytolatex

# Creates a new dataframe for updated question with freeform formula
new_df = pd.DataFrame(columns=['question', 'label'] )

# looping through all data    
for i in range(len(df)):
    new_df.loc[i,'label'] = df.loc[i,'label']
    test = df.loc[i, 'question']
    #regex value for searching all characters from '$' to the next '$' symbol
    check = re.search('[$].+?[$]',test)
    #checks how many instance of latex enclosure are there in the question     
    check_len = len(re.findall('[$].+?[$]',test))
    iteration = 0
    while iteration<check_len:
        iteration += 1
        # concatenate the question string from all characters before formula + formula + all characters after formula
        # formula = re.sub(" ","",str(latex2sympy(r"{}".format(check.group()))))
        # the formula is a converted LaTex to Sympy then removed all white spaces
        # If new dataset is introduced you can print the index of question and the group expression that it returned error.
        # print("change problem in index", i)
        # print(check.group())
        test = test[:check.span()[0]] + " " + re.sub(" ","",str(latex2sympy(r"{}".format(check.group())))) + " " + test[check.span()[1]:]
        #check for the next LaTex Enclosure if any         
        check = re.search('[$].+?[$]',test)
    #the new question is added to index i       
    new_df.loc[i,'question'] = test

print("Preview of converted ")
new_df.head()

Preview of converted 


,question,label
0,It is the sum of the terms of a sequence.,series_and_sequence
1,Find the sum of the first ten terms of the ari...,series_and_sequence
2,Find the sum of the first 25 terms of the arit...,series_and_sequence
3,The sum of the first 10 terms of an arithmetic...,series_and_sequence
4,The third term of an arithmetic sequence is −1...,series_and_sequence


In [7]:
# removing the spaces between from converted freeform expression
new_df = new_df.replace(' \+ ', '+', regex=True)
new_df = new_df.replace(' - ', '-', regex=True)
# We remove the equal sign so that the sympy library would not automatically evaluate the expression
new_df = new_df.replace(' = ', ' ', regex=True)
# cleaning the unnecessary characters
new_df = new_df.replace('\n','', regex=True) #white space
new_df = new_df.replace('\?','', regex=True) #Question marks
new_df = new_df.replace('\'','', regex=True) #quotation marks
new_df = new_df.replace(',','', regex=True)  #comma
new_df = new_df.replace('[\.]','', regex=True) #periods
new_df = new_df.replace('\t', ' ', regex=True) #tabs
new_df = new_df.replace('[;]', ' ', regex=True) #tabs
new_df = new_df.replace(':', ' ', regex=True) #tabs
new_df = new_df.replace(r"\_\_+", "__", regex=True)

# you can call or convert the dataset_freeform dataframe to view the cleaned dataset before tokenizing into expression trees
dataset_freeform = new_df

In [8]:
question_list = new_df.drop(columns=['label']).values.tolist()
question_list[:10]

[['It is the sum of the terms of a sequence'],
 ['Find the sum of the first ten terms of the arithmetic sequence 41016'],
 ['Find the sum of the first 25 terms of the arithmetic sequence 17 22 2732 …'],
 ['The sum of the first 10 terms of an arithmetic sequence is 530 What is the first term if the last term is 80 and the common difference is 2'],
 ['The third term of an arithmetic sequence is −12 and the seventh term is 8 What is the sum of the first 10 terms'],
 ['Find the sum of the first 50 terms of the arithmetic sequence if the first term is 21 and the twentieth term is 154'],
 ['Find the sum of the first eighteen terms of the arithmetic sequence whose nth term is '],
 ['The first term of an arithmetic sequence is 5 the last term is 45 and the sum is 275 Find the number of terms'],
 ['If the first n terms of the arithmetic sequence 20 18 16 are added how many of these terms will be added to get a sum of -100'],
 ['A yaya receives a starting annual salary of Php 3600000 with a year

In [9]:
from sympy import *
from sympy.parsing.sympy_parser import parse_expr
import re

def tokenize_formula(expression):
    try:
        return srepr(parse_expr(expression, evaluate=False))
    except:
        return expression

for i in range(len(question_list)):
    item_question = question_list[i][0]
    tokenize_question_item = item_question.split()
    for x in range(len(tokenize_question_item)):
        check = re.findall('[\+]|[\*]|[-]|[/]|[0-9]',tokenize_question_item[x])
        if len(tokenize_question_item[x]) == 1:
            check = True 
        if check: 
            print("index: ",i)
            print(tokenize_question_item[x])
            text = re.sub("\s|[,]","",str(tokenize_formula(tokenize_question_item[x])))
            print(text)
            text = re.sub("\d+","(d)",text)
            text = re.sub("[(]['].+?['][)]","(x)",text)
#             text = re.sub("Symbol[(]['].+?['][)]","symbol(x)",text)
            text = re.sub('\s+', '', text)
            tokenize_question_item[x] = text

    question_list[i] = ' '.join(tokenize_question_item)

index:  0
a
Symbol('a')
index:  1
41016
Integer(41016)
index:  2
25
Integer(25)
index:  2
17
Integer(17)
index:  2
22
Integer(22)
index:  2
2732
Integer(2732)
index:  2
…
…
index:  3
10
Integer(10)
index:  3
530
Integer(530)
index:  3
80
Integer(80)
index:  3
2
Integer(2)
index:  4
−12
−12
index:  4
8
Integer(8)
index:  4
10
Integer(10)
index:  5
50
Integer(50)
index:  5
21
Integer(21)
index:  5
154
Integer(154)
index:  7
5
Integer(5)
index:  7
45
Integer(45)
index:  7
275
Integer(275)
index:  8
n
Symbol('n')
index:  8
20
Integer(20)
index:  8
18
Integer(18)
index:  8
16
Integer(16)
index:  8
a
Symbol('a')
index:  8
-100
Integer(-100)
index:  9
A
Symbol('A')
index:  9
a
Symbol('a')
index:  9
3600000
Integer(3600000)
index:  9
a
Symbol('a')
index:  9
300000
Integer(300000)
index:  9
8
Integer(8)
index:  10
a
Symbol('a')
index:  10
1000
Integer(1000)
index:  10
1300
Integer(1300)
index:  10
1600
Integer(1600)
index:  10
52
Integer(52)
index:  11
a
Symbol('a')
index:  11
600000
Integer(60

index:  114
80m
80m
index:  114
I
I
index:  114
10th
10th
index:  114
I
I
index:  116
04812
04812
index:  117
3
Integer(3)
index:  117
-2
Integer(-2)
index:  117
-7\dots
-7\dots
index:  118
23
Integer(23)
index:  118
18
Integer(18)
index:  118
13
Integer(13)
index:  118
8
Integer(8)
index:  118
3
Integer(3)
index:  118
-7
Integer(-7)
index:  118
-12
Integer(-12)
index:  119
-7
Integer(-7)
index:  119
-4
Integer(-4)
index:  119
-1
Integer(-1)
index:  119
2
Integer(2)
index:  119
5\dots
5\dots
index:  120
3
Integer(3)
index:  120
13/4
Mul(Integer(13)Pow(Integer(4)Integer(-1)))
index:  120
7/2
Mul(Integer(7)Pow(Integer(2)Integer(-1)))
index:  120
15/4
Mul(Integer(15)Pow(Integer(4)Integer(-1)))
index:  122
a_3
Symbol('a_3')
index:  122
4
Integer(4)
index:  122
a_5
Symbol('a_5')
index:  122
14
Integer(14)
index:  123
7*p+2
Add(Mul(Integer(7)Symbol('p'))Integer(2))
index:  123
5*p+12
Add(Mul(Integer(5)Symbol('p'))Integer(12))
index:  123
3*p+22
Add(Mul(Integer(3)Symbol('p'))Integer(22))
inde

Integer(-1)
index:  209
3
Integer(3)
index:  209
7
Integer(7)
index:  209
11…
11…
index:  210
2-3**(1/2)
Add(Integer(2)Mul(Integer(-1)Pow(Integer(3)Mul(Integer(1)Pow(Integer(2)Integer(-1))))))
index:  210
4-3**(1/2)
Add(Integer(4)Mul(Integer(-1)Pow(Integer(3)Mul(Integer(1)Pow(Integer(2)Integer(-1))))))
index:  211
a_1
Symbol('a_1')
index:  211
-3
Integer(-3)
index:  211
a_5
Symbol('a_5')
index:  211
5
Integer(5)
index:  211
a_{10}
a_{10}
index:  212
a_3
Symbol('a_3')
index:  212
11
Integer(11)
index:  212
a_5
Symbol('a_5')
index:  212
7
Integer(7)
index:  212
-9
Integer(-9)
index:  212
k
Symbol('k')
index:  213
3
Integer(3)
index:  213
8
Integer(8)
index:  213
16
Integer(16)
index:  214
a
Symbol('a')
index:  214
40
Integer(40)
index:  214
95
Integer(95)
index:  214
150
Integer(150)
index:  214
16
Integer(16)
index:  215
5
Integer(5)
index:  215
65
Integer(65)
index:  215
8
Integer(8)
index:  216
a
Symbol('a')
index:  216
a
Symbol('a')
index:  216
P2700000
Symbol('P2700000')
index:  216

index:  284
g(2)
Function('g')(Integer(2))
index:  285
2*x**6+3*x**4-x**2+3
Add(Mul(Integer(2)Pow(Symbol('x')Integer(6)))Mul(Integer(3)Pow(Symbol('x')Integer(4)))Mul(Integer(-1)Pow(Symbol('x')Integer(2)))Integer(3))
index:  285
g(3)
Function('g')(Integer(3))
index:  286
2*x**6+3*x**4-x**2+3
Add(Mul(Integer(2)Pow(Symbol('x')Integer(6)))Mul(Integer(3)Pow(Symbol('x')Integer(4)))Mul(Integer(-1)Pow(Symbol('x')Integer(2)))Integer(3))
index:  286
g*(-1)
Mul(Integer(-1)Symbol('g'))
index:  287
2*x**3-1*7*x+3
Add(Mul(Integer(2)Pow(Symbol('x')Integer(3)))Mul(Integer(-1)Mul(Integer(1)Integer(7)Symbol('x')))Integer(3))
index:  287
f*(-3)
Mul(Integer(-3)Symbol('f'))
index:  288
2*x**3-1*7*x+3
Add(Mul(Integer(2)Pow(Symbol('x')Integer(3)))Mul(Integer(-1)Mul(Integer(1)Integer(7)Symbol('x')))Integer(3))
index:  288
f(5)
Function('f')(Integer(5))
index:  289
2*x**3-1*7*x+3
Add(Mul(Integer(2)Pow(Symbol('x')Integer(3)))Mul(Integer(-1)Mul(Integer(1)Integer(7)Symbol('x')))Integer(3))
index:  289
f*(-10)
Mul

Add(Mul(Integer(2)Pow(Symbol('x')Integer(4)))Mul(Integer(3)Pow(Symbol('x')Integer(2)))Mul(Integer(4)Symbol('x'))Integer(1))
index:  332
\
\
index:  333
5*x**3
Mul(Integer(5)Pow(Symbol('x')Integer(3)))
index:  334
5*x**3
Mul(Integer(5)Pow(Symbol('x')Integer(3)))
index:  335
5*x**3
Mul(Integer(5)Pow(Symbol('x')Integer(3)))
index:  336
5*x**3
Mul(Integer(5)Pow(Symbol('x')Integer(3)))
index:  337
x**6-1*3*x**3+1
Add(Pow(Symbol('x')Integer(6))Mul(Integer(-1)Mul(Integer(1)Integer(3)Pow(Symbol('x')Integer(3))))Integer(1))
index:  338
x**6-1*3*x**3+1
Add(Pow(Symbol('x')Integer(6))Mul(Integer(-1)Mul(Integer(1)Integer(3)Pow(Symbol('x')Integer(3))))Integer(1))
index:  339
x**6-1*3*x**3+1
Add(Pow(Symbol('x')Integer(6))Mul(Integer(-1)Mul(Integer(1)Integer(3)Pow(Symbol('x')Integer(3))))Integer(1))
index:  340
x**6-1*3*x**3+1
Add(Pow(Symbol('x')Integer(6))Mul(Integer(-1)Mul(Integer(1)Integer(3)Pow(Symbol('x')Integer(3))))Integer(1))
index:  341
3*x**3+2*x
Add(Mul(Integer(3)Pow(Symbol('x')Integer(3)))

...

Add(Pow(Symbol('x')Integer(2))Mul(Integer(-1)Mul(Integer(1)Integer(5)Symbol('x')))Integer(6))
index:  736
x-2
Add(Symbol('x')Mul(Integer(-1)Integer(2)))
index:  737
a
Symbol('a')
index:  737
5*x**2+4*x-1
Add(Mul(Integer(5)Pow(Symbol('x')Integer(2)))Mul(Integer(4)Symbol('x'))Mul(Integer(-1)Integer(1)))
index:  737
x+1
Add(Symbol('x')Integer(1))
index:  738
a
Symbol('a')
index:  738
3*x**2-1*5*x-16
Add(Mul(Integer(3)Pow(Symbol('x')Integer(2)))Mul(Integer(-1)Mul(Integer(1)Integer(5)Symbol('x')))Mul(Integer(-1)Integer(16)))
index:  738
x-4
Add(Symbol('x')Mul(Integer(-1)Integer(4)))
index:  739
a
Symbol('a')
index:  739
x**3-1*5*x**2+7*x+7
Add(Pow(Symbol('x')Integer(3))Mul(Integer(-1)Mul(Integer(1)Integer(5)Pow(Symbol('x')Integer(2))))Mul(Integer(7)Symbol('x'))Integer(7))
index:  739
x+1
Add(Symbol('x')Integer(1))
index:  740
a
Symbol('a')
index:  740
-1*5*x**3+2*x**3+7*x+3
Add(Mul(Integer(-1)Integer(5)Pow(Symbol('x')Integer(3)))Mul(Integer(2)Pow(Symbol('x')Integer(3)))Mul(Integer(7)Symbol(

Integer(0)
index:  790
a
Symbol('a')
index:  790
5+5/x**2
Add(Integer(5)Mul(Integer(5)Pow(Pow(Symbol('x')Integer(2))Integer(-1))))
index:  790
0
Integer(0)
index:  791
a
Symbol('a')
index:  791
10**x+2*x+1
Add(Pow(Integer(10)Symbol('x'))Mul(Integer(2)Symbol('x'))Integer(1))
index:  791
0
Integer(0)
index:  792
a
Symbol('a')
index:  792
10-x**2
Add(Integer(10)Mul(Integer(-1)Pow(Symbol('x')Integer(2))))
index:  792
0
Integer(0)
index:  793
a
Symbol('a')
index:  793
1*x**2/2-1
Add(Mul(Integer(1)Pow(Integer(2)Integer(-1))Pow(Symbol('x')Integer(2)))Mul(Integer(-1)Integer(1)))
index:  793
0
Integer(0)
index:  794
a
Symbol('a')
index:  794
3*x**4*y**3/4-1*21*x
Add(Mul(Integer(3)Pow(Integer(4)Integer(-1))Pow(Symbol('x')Integer(4))Pow(Symbol('y')Integer(3)))Mul(Integer(-1)Mul(Integer(1)Integer(21)Symbol('x'))))
index:  794
0
Integer(0)
index:  795
a
Symbol('a')
index:  795
x**(1/2)-3+9
Add(Pow(Symbol('x')Mul(Integer(1)Pow(Integer(2)Integer(-1))))Mul(Integer(-1)Integer(3))Integer(9))
index:  795

0
Integer(0)
index:  859
j**2-359*j-360
Add(Pow(Symbol('j')Integer(2))Mul(Integer(-1)Mul(Integer(359)Symbol('j')))Mul(Integer(-1)Integer(360)))
index:  860
u
Symbol('u')
index:  860
-3*u**3/2-51*u**2-867*u/2
Add(Mul(Integer(-1)Integer(3)Pow(Integer(2)Integer(-1))Pow(Symbol('u')Integer(3)))Mul(Integer(-1)Mul(Integer(51)Pow(Symbol('u')Integer(2))))Mul(Integer(-1)Mul(Integer(867)Pow(Integer(2)Integer(-1))Symbol('u'))))
index:  860
0
Integer(0)
index:  861
-2*u**2+54*u-100
Add(Mul(Integer(-1)Integer(2)Pow(Symbol('u')Integer(2)))Mul(Integer(54)Symbol('u'))Mul(Integer(-1)Integer(100)))
index:  862
-o**3/9+7*o**2/9+40*o/9+44/9
Add(Mul(Integer(7)Pow(Integer(9)Integer(-1))Pow(Symbol('o')Integer(2)))Mul(Integer(40)Pow(Integer(9)Integer(-1))Symbol('o'))Mul(Pow(Integer(9)Integer(-1))Mul(Integer(-1)Pow(Symbol('o')Integer(3))))Mul(Integer(44)Pow(Integer(9)Integer(-1))))
index:  863
l
Symbol('l')
index:  863
-2156*l**5+6132*l**4-4152*l**3+1072*l**2-96*l
Add(Mul(Integer(-1)Integer(2156)Pow(Symbol('l')

Add(Mul(Integer(655)Pow(Integer(6)Integer(-1))Pow(Symbol('m')Integer(3)))Mul(Integer(3935)Pow(Integer(3)Integer(-1))Pow(Symbol('m')Integer(2)))Mul(Integer(3950)Symbol('m'))Integer(60))
index:  895
0
Integer(0)
index:  896
o
Symbol('o')
index:  896
-58*o**2/3+10*o/3+16
Add(Mul(Integer(-1)Integer(58)Pow(Integer(3)Integer(-1))Pow(Symbol('o')Integer(2)))Mul(Integer(10)Pow(Integer(3)Integer(-1))Symbol('o'))Integer(16))
index:  896
0
Integer(0)
index:  897
-3*b**4/8+21*b**3/8-39*b**2/8-9*b/8+27/4
Add(Mul(Integer(-1)Integer(3)Pow(Integer(8)Integer(-1))Pow(Symbol('b')Integer(4)))Mul(Integer(21)Pow(Integer(8)Integer(-1))Pow(Symbol('b')Integer(3)))Mul(Integer(-1)Mul(Integer(39)Pow(Integer(8)Integer(-1))Pow(Symbol('b')Integer(2))))Mul(Integer(-1)Mul(Integer(9)Pow(Integer(8)Integer(-1))Symbol('b')))Mul(Integer(27)Pow(Integer(4)Integer(-1))))
index:  897
0
Integer(0)
index:  897
b
Symbol('b')
index:  898
28*d**3+116*d**2+16*d
Add(Mul(Integer(28)Pow(Symbol('d')Integer(3)))Mul(Integer(116)Pow(Symbol(

Add(Mul(Integer(-1)Integer(7)Pow(Integer(5)Integer(-1))Pow(Symbol('v')Integer(5)))Mul(Integer(-1)Mul(Integer(47)Pow(Integer(5)Integer(-1))Pow(Symbol('v')Integer(4))))Mul(Integer(-1)Mul(Integer(84)Pow(Integer(5)Integer(-1))Pow(Symbol('v')Integer(3))))Mul(Integer(-1)Mul(Integer(28)Pow(Integer(5)Integer(-1))Pow(Symbol('v')Integer(2))))Mul(Integer(16)Pow(Integer(5)Integer(-1))Symbol('v')))
index:  932
0
Integer(0)
index:  932
v
Symbol('v')
index:  933
a
Symbol('a')
index:  933
172*a**2-524*a+24
Add(Mul(Integer(172)Pow(Symbol('a')Integer(2)))Mul(Integer(-1)Mul(Integer(524)Symbol('a')))Integer(24))
index:  933
0
Integer(0)
index:  934
-2*p**3/3+22*p**2/3-20*p/3
Add(Mul(Integer(-1)Integer(2)Pow(Integer(3)Integer(-1))Pow(Symbol('p')Integer(3)))Mul(Integer(22)Pow(Integer(3)Integer(-1))Pow(Symbol('p')Integer(2)))Mul(Integer(-1)Mul(Integer(20)Pow(Integer(3)Integer(-1))Symbol('p'))))
index:  935
r
Symbol('r')
index:  935
10*r**3+235*r**2+510*r-315
Add(Mul(Integer(10)Pow(Symbol('r')Integer(3)))Mul(

In [17]:
labels = new_df.loc[:,'label'].values.tolist()
# print(labels)
# print(question_list)
# generated_dataset = pd.DataFrame([question_list, labels], columns=['question', 'label'])
# generated_dataset.to_csv('dataset_after_transformation.csv', encoding='utf-8')

In [18]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

words = stopwords.words('english')

vect = CountVectorizer(token_pattern = '[a-zA-Z0-9,=+-<>()_{}]+',stop_words = words, min_df=0)
vect.fit(question_list)

CountVectorizer(min_df=0,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                token_pattern='[a-zA-Z0-9,=+-<>()_{}]+')

In [14]:
# len(labels)

In [19]:
len(question_list)

954

In [20]:
lab_ques = {'question':list(question_list), 'label':list(labels)}
generated_dataset = pd.DataFrame(lab_ques)
# generated_dataset.to_csv('dataset_after_transformation.csv', encoding='utf-8')
generated_dataset

,question,label
0,It is the sum of the terms of Symbol(x) sequence,series_and_sequence
1,Find the sum of the first ten terms of the ari...,series_and_sequence
2,Find the sum of the first Integer((d)) terms o...,series_and_sequence
3,The sum of the first Integer((d)) terms of an ...,series_and_sequence
4,The third term of an arithmetic sequence is −(...,series_and_sequence
...,...,...
949,Factor Add(Mul(Integer(-(d))Integer((d))Pow(Sy...,polynomial_equation
950,Factor Add(Mul(Integer(-(d))Integer((d))Pow(In...,polynomial_equation
951,What is Symbol(x) in Add(Mul(Integer((d))Pow(S...,polynomial_equation
952,Find Symbol(x) given that Add(Mul(Integer((d))...,polynomial_equation


In [ ]:
# generated_dataset.to_csv('dataset_after_transformation.csv', encoding='utf-8')

In [21]:
print("Column Names and their count")
print(format(vect.vocabulary_))


Column Names and their count
{'sum': 822, 'terms': 833, 'symbol(x)': 825, 'sequence': 785, 'find': 437, 'first': 440, 'ten': 829, 'arithmetic': 299, 'integer((d))': 496, 'term': 832, 'last': 513, 'common': 336, 'difference': 377, 'third': 837, '(d)': 1, 'seventh': 792, 'twentieth': 852, 'eighteen': 400, 'whose': 879, 'nth': 662, 'number': 663, 'added': 281, 'many': 535, 'get': 461, 'integer(-(d))': 497, 'yaya': 888, 'receives': 739, 'starting': 811, 'annual': 290, 'salary': 771, 'php': 696, 'yearly': 890, 'increase': 484, 'total': 847, 'income': 483, 'years': 891, 'jane': 500, 'saving': 775, 'pair': 681, 'shoes': 793, 'weekly': 875, 'allowance': 286, 'able': 22, 'save': 772, 'week': 874, 'second': 782, 'continued': 348, 'pattern': 684, 'made': 531, 'deposits': 370, 'much': 560, 'mary': 536, 'gets': 462, 'monthly': 554, 'annually': 291, 'receive': 738, 'three': 839, 'mirasol': 548, 'saved': 773, 'day': 364, 'january': 501, 'month': 553, 'end': 403, 'mrs': 559, 'de': 365, 'la': 512, 'cru

In [22]:
print("Total Columns")
print(len(vect.get_feature_names_out()))

Total Columns
894


In [23]:
bag_of_words = vect.transform(question_list)
format(bag_of_words.toarray())

'[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]]'

In [24]:
import scipy.sparse

bow_Matrix = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns = vect.get_feature_names_out())
X = bow_Matrix.values
y = new_df.loc[:,"label"].values
bow_Matrix.head()

,(binomial),(d),(d)+(d)+(d)+,(d)cm,(d)m,(d)n,(d)nd,(d)rd,(d)s_(d),(d)st,...,worth,would,write,x,yaya,year,yearly,years,zero,zeros
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state= 0)

In [26]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100,random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [27]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[77  2  3]
 [ 6 79  0]
 [ 2  0 70]]
                     precision    recall  f1-score   support

polynomial_equation       0.91      0.94      0.92        82
        polynomials       0.98      0.93      0.95        85
series_and_sequence       0.96      0.97      0.97        72

           accuracy                           0.95       239
          macro avg       0.95      0.95      0.95       239
       weighted avg       0.95      0.95      0.95       239



In [28]:
accuracy_score(y_test, y_pred)

0.9456066945606695

Importing the Model and Vectorizer using Pickle

In [ ]:
import pickle
## Uncomment to produce the file
## save the model to disk
# model = 'rfa_model_v1.sav'
# pickle.dump(classifier, open(model, 'wb'))
# myvectorizer = 'vectorizer_v1.sav'
# pickle.dump(vect, open(myvectorizer, 'wb'))

In [ ]:
create_excel = bow_Matrix
create_excel= create_excel.assign(ML_class=y)
print(create_excel)

# CREATING A DATASET

# create_excel.to_csv("BOW_dataset_v1.csv", encoding='utf-8') 